# Grover's Algorithm for Protein Molecule Selection

This project leverages a simulated quantum computing circuit to apply Grover's algorithm, aiming to demonstrate the ability to identify the protein molecule with the highest binding affinity. This method has the potential to enhance the development of more effective medications for various diseases.

# Theory

* **Qubits**: The fundamental unit of quantum information, analogous to bits in classical computers. Unlike bits that are restricted to either 0 or 1, qubits can exist in a **superposition** of both states simultaneously.

* **State Vector**: Describes the state of a qubit using a complex number formula:

$$ \hat{v} = \hat{0}\alpha + \hat{1}\beta$$

* $ \alpha $ and $ \beta $ are complex numbers representing **probability amplitudes**, indicating the likelihood of the qubit being in the $ \hat{0} $ or $ \hat{1} $ state.

* **Oracles**: conceptual tool in quantum computing that identifies specific solutions without directly knowing them. It operates by analyzing the nature of a given state.

    * If a state $ \hat{v} $ satisfies the oracle's condition, the phase of $ \hat{v} $ is flipped.
    
    * If a state $ \hat{y} $ doesn't satisfy the oracle's condition, its phase remains unchanged.

**Grover's algorithm** is a quantum search algorithm designed to find the correct solution from a set of possibilities. The steps are as follows:

1. Assign a qubit to each possibility in the set.
2. Initialize all qubits in a superposition of all possible states.
3. Apply the oracle to flip the phase of the correct solution.
4. Use a diffusion operator to amplify the amplitude of the flipped solution, making it more prominent.
5. Repeat Steps 3 and 4 a specific number of times.
6. Measure the final state, which will reveal the solution with high probability.

# Prerequisites

To get started, the Qiskit packages provided by IBM are installed, which allow for the simulation of quantum computing environments. They are then imported into the environment.

In [1]:
!pip install qiskit
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 71.8 MB/s eta 0:00:00


In [2]:
from qiskit import QuantumCircuit, transpile, assemble
import csv
import numpy as np
import pandas as pd
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector

# Import Dataset

To concentrate on the core concepts of quantum computing, a simplified dataset was utilized. This dataset comprises 10 molecules, each characterized by its molecular weight, binding affinity, and hydrophobicity. The objective is to employ Grover's algorithm to identify the molecule exhibiting the highest binding affinity.

In [3]:
df = pd.read_csv('/kaggle/input/protein-molecules-dataset/molecules.csv')
df

,Molecule,MolecularWeight,BindingAffinity,Hydrophobicity
0,Molecule1,120.5,7.8,3.2
1,Molecule2,150.8,8.3,4.1
2,Molecule3,180.3,9.1,5.6
3,Molecule4,210.2,6.7,2.8
4,Molecule5,175.5,9.5,5.0
5,Molecule6,190.0,8.9,4.7
6,Molecule7,200.5,7.2,3.5
7,Molecule8,185.2,9.8,5.4
8,Molecule9,195.6,7.9,3.9
9,Molecule10,205.8,6.5,2.6


The following function performs a couple of tasks:
* Loads and iterates over each row in the CSV dataset.
* The following numerical features are extracted: MolecularWeight, BindingAffinity, and Hydrophobicity.
* The numerical features are all normalized on a scale from 0 to 255.
* All 3 normalizations are concatenated and converted to an 8-bit binary string.
* A tuple containing the binary string and binding affinity are appended to a list.

In [4]:
def load_and_normalize_molecules(filename):
    with open(filename, mode='r') as file:
        reader = csv.DictReader(file)
        molecules = []
        for row in reader:
            molecular_weight = float(row['MolecularWeight']) / 1000
            binding_affinity = float(row['BindingAffinity']) / 10
            hydrophobicity = float(row['Hydrophobicity']) / 10
            molecular_weight_binary = format(int(molecular_weight * 255), '08b')
            binding_affinity_binary = format(int(binding_affinity * 255), '08b')
            hydrophobicity_binary = format(int(hydrophobicity * 255), '08b')
            binary_string = molecular_weight_binary + binding_affinity_binary + hydrophobicity_binary
            molecules.append((binary_string, float(row['BindingAffinity'])))
    return molecules

# Key Functions
The following functions help construct our quantum search algorithm. The first function determines the most optimal molecule in the list. The most optimal molecule is one with the highest binding affinity. The following function converts a binary string to an integer. The final function initializes the circuit by applying the Hadamard gate to impose a superposition to all qubits.

In [5]:
def find_optimal_molecule(molecules):
    return max(molecules, key=lambda x: x[1])[0]

def binary_to_integer(binary_string):
    return int(binary_string, 2)

def initialize_circuit(quantum_circuit, qubits):
    quantum_circuit.h(qubits)
    return quantum_circuit

The following function implements a quantum circuit that effectively amplifies the amplitude of the target state, making it more likely to be measured in subsequent quantum measurements. This oracle is a key component of Grover's algorithm, which can be used to efficiently search for molecules with desired properties within a large dataset.

1. Target State Representation:

The target_state is converted into a binary string. This binary string represents the desired quantum state that the oracle aims to amplify.

2. Qubit Preparation:

The function iterates over the binary string and applies the **Pauli-X gate** to qubits corresponding to $ \hat{0} $ bits. This step effectively prepares the qubits to represent the target state. In essence, it flips the qubits that need to be in the $ \hat{1} $ state to match the target state.

3. Grover Diffusion Operator:

The Grover diffusion operator is applied to amplify the amplitude of the target state while reducing the amplitudes of other states. This operator consists of three main steps:
* **Hadamard Transformation**: A Hadamard gate is applied to the last qubit, creating a superposition. This step is essential for the subsequent multi-controlled X gate to function correctly.
* **Multi-Controlled X Gate**: A multi-controlled X gate is applied to all qubits except the last one, with the last qubit as the target qubit. This gate flips the target qubit if all the control qubits are in the $ \hat{1} $ state. In this case, the control qubits are the ones that have been prepared to represent the target state.
* Hadamard Transformation: Another Hadamard gate is applied to the last qubit. This completes the Grover diffusion operator and amplifies the amplitude of the target state.

4. Undoing Preparation:

The function applies the Pauli-X gate to qubits corresponding to $ \hat{0} $ bits in the binary string again. This step undoes the initial qubit preparation, ensuring that the oracle correctly flips the amplitude of the target state without affecting other states.

In [6]:
def create_oracle(quantum_circuit, qubits, target_state):
    qubits = list(qubits)
    binary_string = format(target_state, f"0{len(qubits)}b")
    
    for index, bit in enumerate(binary_string):
        if bit == '0':
            quantum_circuit.x(qubits[index])
    
    # Grover Diffusion Operator
    quantum_circuit.h(qubits[-1])
    quantum_circuit.mcx(qubits[:-1], qubits[-1])
    quantum_circuit.h(qubits[-1])

    for index, bit in enumerate(binary_string):
        if bit == '0':
            quantum_circuit.x(qubits[index])

    return quantum_circuit

The following function implements the Grover diffusion operator within a quantum circuit.

1. Applies a Hadamard gate to all qubits.
2. Applies the Pauli-X gate to all qubits.
3. Applies a Hadamard gate to the last qubit.
4. Applies a multi-controlled X gate with the last qubit as the target and the rest as controls.
5. Applies a Hadamard gate to the last qubit.
6. Applies the Pauli-X gate to all qubits.
7. Applies a Hadamard gate to all qubits.
8. Returns the modified quantum circuit.

In [7]:
def diffusion_operator(quantum_circuit, qubits):
    qubits = list(qubits)
    quantum_circuit.h(qubits)
    quantum_circuit.x(qubits)
    quantum_circuit.h(qubits[-1])
    quantum_circuit.mcx(qubits[:-1], qubits[-1])
    quantum_circuit.h(qubits[-1])
    quantum_circuit.x(qubits)
    quantum_circuit.h(qubits)
    return quantum_circuit

# Grover's Algorithm

As a reminder, Grover's algorithm is as follows:

1. Assign a qubit to each possibility in the set.
2. Initialize all qubits in a superposition of all possible states.
3. Apply the oracle to flip the phase of the correct solution.
4. Use a diffusion operator to amplify the amplitude of the flipped solution, making it more prominent.
5. Repeat Steps 3 and 4 a specific number of times.
6. Measure the final state, which will reveal the solution with high probability.

The following function simply applies the key functions to execute a successful quantum search.

In [8]:
def grovers_search(filename):
    
    # Step 1
    molecules = load_and_normalize_molecules(filename)
    optimal_molecule = find_optimal_molecule(molecules)
    target_state = binary_to_integer(optimal_molecule)
    n_qubits = len(optimal_molecule)
    quantum_circuit = QuantumCircuit(n_qubits)
    
    # Step 2
    quantum_circuit = initialize_circuit(quantum_circuit, range(n_qubits))
    
    # Step 3
    quantum_circuit = create_oracle(quantum_circuit, range(n_qubits), target_state)
    
    # Step 4
    quantum_circuit = diffusion_operator(quantum_circuit, range(n_qubits))
    
    # Step 6
    statevector_simulator = Aer.get_backend('statevector_simulator')
    quantum_circuit = transpile(quantum_circuit, statevector_simulator)
    result = statevector_simulator.run(quantum_circuit).result()
    statevector = result.get_statevector()
    print(f"Optimal molecule (binary): {optimal_molecule}")
    print(f"Optimal molecule (decimal): {target_state}")
    print("Statevector probabilities:")
    print(statevector)

# Output

The following is the output of the Grover's algorithm. 

* Optimal molecule (binary): This represents the binary string of the molecule identified as having the highest binding affinity.
* Optimal molecule (decimal): The decimal equivalent of the optimal molecule's binary string.
* Statevector probabilities: An array of complex numbers representing the probability amplitudes of each possible state of the quantum system. The indices of the array correspond to the different states, and the values represent the probability amplitudes. The dims attribute specifies the dimensions of the statevector, which is a $2^n$-dimensional vector, where n is the number of qubits in the system.

In [9]:
grovers_search('/kaggle/input/protein-molecules-dataset/molecules.csv')

Optimal molecule (binary): 001011111111100110001001
Optimal molecule (decimal): 3144073
Statevector probabilities:
Statevector([-0.00024414-1.01655228e-18j, -0.00024414-1.04645087e-18j,
             -0.00024414-1.04645087e-18j, ..., -0.00024414-1.67432139e-18j,
             -0.00024414-1.67432139e-18j, -0.00024414-1.70421999e-18j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2))
